In [8]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [9]:
dataset_path = r'C:\Users\gowth\OneDrive\Desktop\HandGesture\leapGestRecog'  

image_size = (128, 126)  
num_channels = 1


In [10]:
images = []
labels = []

for top_folder in os.listdir(dataset_path):
    top_folder_path = os.path.join(dataset_path, top_folder)
    if os.path.isdir(top_folder_path):  
        for gesture_label in os.listdir(top_folder_path):
            gesture_path = os.path.join(top_folder_path, gesture_label)
            if os.path.isdir(gesture_path):  
                for image_name in os.listdir(gesture_path):
                    image_path = os.path.join(gesture_path, image_name)
                    image = cv2.imread(image_path)

                    if num_channels == 1:
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                    image = cv2.resize(image, image_size)

                    image = image / 255.0

                    images.append(image)
                    labels.append(gesture_label)

images = np.array(images)
labels = np.array(labels)

if num_channels == 1:
    images = np.expand_dims(images, axis=-1)

In [11]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

labels_categorical = to_categorical(labels_encoded, num_classes=len(np.unique(labels_encoded)))

print("Images shape:", images.shape)
print("Labels shape:", labels_categorical.shape)

Images shape: (20000, 126, 128, 1)
Labels shape: (20000, 10)
